[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ramonzaca/MLSecOPs/blob/main/TP_04/04_bias_and_explainability.ipynb)

**What's the model doing? - Practice 4**

*The model is working and we are keeping track of the data and the model's performance.*

*Now, we want to understand what the model is doing. Does it have biases? Are the predictions fair? We want to understand the model's predictions and the model's behavior.*

*We will use [SHAP](https://shap.readthedocs.io/en/latest/index.html) to do so.*


![ShapValues](https://shap.readthedocs.io/en/latest/_images/shap_header.png)

---

In [ ]:
# First, let's install Shap if it's not already installed
try:
    import shap
except ImportError:
    !pip install shap

In [ ]:
# Suppress warnings for cleaner output
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [ ]:
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import shap
from sklearn import linear_model, tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

---

*In this first part we will use the diabetes dataset to create some models and then explain those models' predictions.*

In [ ]:
# 1. Data Preparation
print("1. Data Preparation")

X, y = shap.datasets.diabetes()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# rather than use the whole training set to estimate expected values, we summarize with
# a set of weighted kmeans, each weighted by the number of points they represent.
X_train_summary = shap.kmeans(X_train, 10)


def print_accuracy(f):
    print(
        f"Root mean squared test error = {np.sqrt(np.mean((f(X_test) - y_test) ** 2))}"
    )
    time.sleep(0.5)  # to let the print get out before any progress bars


shap.initjs()

---

*Before going further, let's compute a hierarchical clustering of the input features*

In [ ]:
partition_tree = shap.utils.partition_tree(X)
plt.figure(figsize=(15, 6))
sp.cluster.hierarchy.dendrogram(partition_tree, labels=X.columns)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("feature")
plt.ylabel("distance")
plt.show()

---

*Let's begin with a rather basic model, a linear regression model.*

In [ ]:
lin_regr = linear_model.LinearRegression()
lin_regr.fit(X_train, y_train)

print_accuracy(lin_regr.predict)

*Explain a single prediction from the test set*

In [ ]:
# First create the explainer object (Notice that we use the summary of the training set as the background dataset)
ex = shap.KernelExplainer(lin_regr.predict, X_train_summary)

In [ ]:
# Let's explain the prediction of the first sample in the test set
shap_values = ex.shap_values(X_test.iloc[0, :])
shap.force_plot(ex.expected_value, shap_values, X_test.iloc[0, :])

*Explain all the predictions in the test set*

In [ ]:
shap_values = ex.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
# Let's see the dependence of the bmi feature with regards to the age
shap.dependence_plot("bmi", shap_values, X_test)

In [ ]:
# Let's see the force plot of all the samples in the test set
shap.force_plot(ex.expected_value, shap_values, X_test)

---

*Let's try a decision tree regressor*

In [ ]:
dtree = tree.DecisionTreeRegressor(min_samples_split=20)
dtree.fit(X_train, y_train)
print_accuracy(dtree.predict)

# explain all the predictions in the test set
ex = shap.TreeExplainer(dtree)
shap_values = ex.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.dependence_plot("bmi", shap_values, X_test)

In [ ]:
shap.force_plot(ex.expected_value, shap_values, X_test)

*Let's try a RandomForest*

*In this case, we will use the fast `TreeExplainer` implementation.*

In [ ]:
rforest = RandomForestRegressor(
    n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0
)
rforest.fit(X_train, y_train)
print_accuracy(rforest.predict)

# explain all the predictions in the test set
explainer = shap.TreeExplainer(rforest)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.dependence_plot("bmi", shap_values, X_test)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test)

*Finally, let's train a Neural network*

In [ ]:
nn = MLPRegressor(solver="lbfgs", alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=0)
nn.fit(X_train, y_train)
print_accuracy(nn.predict)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(nn.predict, X_train_summary)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.dependence_plot("bmi", shap_values, X_test)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test)

---

*How representative is each explainer with regards to a generic one?*

In [ ]:
instance = X[0:1]


# build a masker from partition tree
masker = shap.maskers.Partition(X, clustering=partition_tree)

# build explainer objects
raw_explainer = shap.PartitionExplainer(rforest.predict, X)
masker_explainer = shap.PartitionExplainer(rforest.predict, masker)

# compute SHAP values
raw_shap_values = raw_explainer(instance)
masker_shap_values = masker_explainer(instance)

In [ ]:
# comparison the masker and the original data sizes
print(f"X size: {sys.getsizeof(X)/1024:.2f} kB")
print(f"masker size: {sys.getsizeof(masker)} B")

*Compare to Tree SHAP*

In [ ]:
tree_explainer = shap.TreeExplainer(rforest, X)
tree_shap_values = tree_explainer(instance)

plt.figure(figsize=(15, 6))
plt.plot(tree_shap_values[0].values, label="Tree SHAP")
plt.plot(masker_shap_values[0].values, "g--", label="Partition SHAP")
plt.plot(raw_shap_values[0].values, "r--", label="Raw SHAP")

plt.legend()
plt.show()

Partition SHAP values using a partition tree are nice estimation of SHAP values. The partition tree is a good way to reduce the number of input features and speed up the computation.

*Another way to plot the explaination of the instance*

In [ ]:
shap.plots.waterfall(masker_shap_values[0])

---

*Now let's try to do a classification analysis. For that we will use the diabetes dataset but classifying whether the person has diabetes or not based on the mean of the dataset.*

*(This is for educational purposes only. This is not a good way to do a diabetes classification :)*


In [ ]:
y_train_classification = (y_train < y_train.mean()).astype(int)
y_test_classification = (y_test < y_train.mean()).astype(int)

In [ ]:
# Build the model
rf_clf = RandomForestClassifier(max_features=2, n_estimators=50, bootstrap=True)

rf_clf.fit(X_train, y_train_classification)

# Make prediction on the testing data
y_pred = rf_clf.predict(X_test)

# Classification Report
print(classification_report(y_pred, y_test_classification))

In [ ]:
# explain all the predictions in the test set for the negative class
# (Notice the time difference between sampling and using the full dataset)
explainer = shap.KernelExplainer(rf_clf.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[..., 0], X_test)

In [ ]:
# Explain all the predictions in the test set for the positive class. Notice something weird?
shap.force_plot(explainer.expected_value[1], shap_values[..., 1], X_test)